## 1. Data Preparation

### Load data

In [1]:
import pandas as pd

df = pd.read_csv("whole Dataset.csv")

In [2]:
df[:5]

,nodule_name,nodule_category,calcification,internal_structure,lobulation,margin,nodule_type,sphericity,texture,global_seed,slice_thickness,slices_present,file_path
0,Small Cell Lung Cancer (SCLC),Malignant,Absent,Soft Tissue,Nn-Mk,P-Sharp,m4,Lin-Ov,NS-PS,3388.5178,2.00,"[53, 54, 55, 56, 57]",../20241221_074106\20241221_074106\LIDC-IDRI\L...
1,Lymphoma,Malignant,Absent,Soft Tissue,N-Marked,Poo-Sh,m7,Ovoid,Part Solid/Mixed,9335.7762,2.00,"[70, 71, 72, 73, 74, 75]",../20241221_074106\20241221_074106\LIDC-IDRI\L...
2,Bronchioloalveolar Hyperplasia,Benign,Absent,Soft Tissue,Nn-Mk,Poorly,b6,Linear,NS-PS,9151.7012,2.50,"[26, 27, 28]",../20241221_074106\20241221_074106\LIDC-IDRI\L...
3,Carcinoid Tumors,Malignant,Non-Central,Soft Tissue,NaN,P-Sharp,m5,Ov-Ro,Solid,201.8990,2.50,"[51, 52, 53]",../20241221_074106\20241221_074106\LIDC-IDRI\L...
4,Intrapulmonary Lymph Nodes,Benign,Absent,Soft Tissue,NaN,Sharp,b7,Ovoid,Solid,8747.3172,1.25,"[23, 24, 25, 26, 27]",../20241221_074106\20241221_074106\LIDC-IDRI\L...


In [3]:
print(len(df))

1487


### Remove 34 nodules with missing annotations

In [4]:
print(df.dtypes)

nodule_name            object
nodule_category        object
calcification          object
internal_structure     object
lobulation             object
margin                 object
nodule_type            object
sphericity             object
texture                object
global_seed           float64
slice_thickness       float64
slices_present         object
file_path              object
dtype: object


In [5]:
import ast

df["slices_present"] = df["slices_present"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [6]:
from collections import Counter

length_counts = Counter(len(x) for x in df["slices_present"])
for k in sorted(length_counts):
    print(f"{k} slices: {length_counts[k]} nodules")

0 slices: 34 nodules
1 slices: 18 nodules
2 slices: 239 nodules
3 slices: 376 nodules
4 slices: 196 nodules
5 slices: 163 nodules
6 slices: 142 nodules
7 slices: 123 nodules
8 slices: 88 nodules
9 slices: 50 nodules
10 slices: 15 nodules
11 slices: 16 nodules
12 slices: 6 nodules
13 slices: 12 nodules
14 slices: 7 nodules
18 slices: 1 nodules
19 slices: 1 nodules


In [7]:
df = df[df['slices_present'].apply(lambda x: len(x) > 0)]

In [8]:
print((df['slices_present'].apply(len) == 0).sum())  

0


In [9]:
print(len(df))

1453


### Encoding Labels

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df['nodule_name'])

In [11]:
label_map = dict(zip(le.classes_, le.transform(le.classes_)))
print(label_map)

{'Active Infection': 0, 'Adenocarcinoma': 1, 'Adenoid Cystic Carcinoma': 2, 'Bronchioloalveolar Hyperplasia': 3, 'Carcinoid Tumors': 4, 'Granuloma': 5, 'Hamartoma': 6, 'Intrapulmonary Lymph Nodes': 7, 'Large Cell (Undifferentiated) Carcinoma': 8, 'Lymphoma': 9, 'Metastatic Tumors': 10, 'Sarcoidosis': 11, 'Sarcomatoid Carcinoma': 12, 'Small Cell Lung Cancer (SCLC)': 13, 'Squamous Cell Carcinoma': 14}


NOTE: Long covid is not part of this study

# 2. Volume input preprocessing

For each sample:

- volume_local: cropped region centered around nodule (based on segmentation)

- volume_context: full slice (resized to something like 256×256), no crop

In [12]:
# Parse annotaions for bounding box

import json

def load_annotations(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    annotations = data.get("annotation", [])
    return annotations


In [13]:
# Get nodule centroid

import numpy as np

def polygon_centroid(norm_polygon, img_shape):
    h, w = img_shape
    x_coords = norm_polygon[::2]
    y_coords = norm_polygon[1::2]
    
    # Convert to pixel space
    x_pixels = [x * w for x in x_coords]
    y_pixels = [y * h for y in y_coords]

    x_center = int(np.mean(x_pixels))
    y_center = int(np.mean(y_pixels))

    return x_center, y_center

In [14]:
# Get local crop and global context

import pydicom
import cv2

def process_slice_local_context(dcm_path, segmentation, local_size=(64, 64), context_size=(256, 256)):
    dcm = pydicom.dcmread(dcm_path)
    img = dcm.pixel_array
    h, w = img.shape

    # Normalize pixel values
    img = img.astype(np.float32)
    img = (img - img.min()) / (img.max() - img.min() + 1e-5)

    x_center, y_center = polygon_centroid(segmentation, (h, w))

    # Crop around the center for local view
    half_h, half_w = local_size[0] // 2, local_size[1] // 2
    y1 = max(y_center - half_h, 0)
    y2 = min(y_center + half_h, h)
    x1 = max(x_center - half_w, 0)
    x2 = min(x_center + half_w, w)
    cropped_local = img[y1:y2, x1:x2]

    # Pad if needed
    cropped_local = cv2.resize(cropped_local, local_size)

    # Resize whole slice for context
    resized_context = cv2.resize(img, context_size)

    return cropped_local, resized_context

In [15]:
import os
import re

def remove_leading_duplicate_folder(path):
    """
    Removes the first repeated folder name at the beginning of a path.
    E.g., '../X\\X\\...' → 'X\\...'
    """
    match = re.match(r"^(?:\.\.[\\/])?([^\\/]+)[\\/]\1[\\/](.+)", path)
    if match:
        return f"{match.group(1)}\\{match.group(2)}"
    return path

In [16]:
df['file_path'] = df['file_path'].apply(remove_leading_duplicate_folder)

In [17]:
# Trying with 1 data sample

json_path = df.iloc[0]['file_path'] 


annotations = load_annotations(json_path)

local_slices = []
context_slices = []

for slice_dict in annotations:
    for slice_idx, data in slice_dict.items():
        dcm_filename = os.path.basename(data["dicomFile"])
        dcm_dir = os.path.dirname(json_path)
        dcm_path = os.path.join(dcm_dir, dcm_filename)
        segmentation = data["segmentation"][0]  
        local, context = process_slice_local_context(
            dcm_path, segmentation,
            local_size=(64, 64),
            context_size=(256, 256)
        )
        local_slices.append(local)
        context_slices.append(context)

In [18]:
# Pads or crops a list of 2D slices to a fixed 3D volume.

def pad_or_crop_volume(slices, target_depth, shape):
    D = len(slices)
    h, w = shape

    if D == 0:
        return np.zeros((1, target_depth, h, w), dtype=np.float32)

    volume = np.stack(slices, axis=0)

    if D < target_depth:
        pad_before = (target_depth - D) // 2
        pad_after = target_depth - D - pad_before
        volume = np.pad(volume, ((pad_before, pad_after), (0, 0), (0, 0)), mode='constant')
    elif D > target_depth:
        start = (D - target_depth) // 2
        volume = volume[start:start+target_depth]

    return volume[None, ...]  # Add channel dim: [1, D, H, W]


In [19]:
volume_local = pad_or_crop_volume(local_slices, target_depth=5, shape=(64, 64))
volume_context = pad_or_crop_volume(context_slices, target_depth=7, shape=(256, 256))

print("volume_local shape:", volume_local.shape)
print("volume_context shape:", volume_context.shape)

volume_local shape: (1, 5, 64, 64)
volume_context shape: (1, 7, 256, 256)


In [20]:
# Applying to entire dataset

from tqdm import tqdm

all_locals = []
all_contexts = []
all_labels = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing samples"):
    try:
        json_path = row['file_path']
        label = row['label']
        annotations = load_annotations(json_path)

        local_slices = []
        context_slices = []

        for slice_dict in annotations:
            for slice_idx, data in slice_dict.items():
                
                dcm_filename = os.path.basename(data["dicomFile"])
                dcm_dir = os.path.dirname(json_path)
                dcm_path = os.path.join(dcm_dir, dcm_filename)

                segmentation = data["segmentation"][0]
                local, context = process_slice_local_context(
                    dcm_path, segmentation,
                    local_size=(64, 64),
                    context_size=(256, 256)
                )
                local_slices.append(local)
                context_slices.append(context)

        volume_local = pad_or_crop_volume(local_slices, target_depth=5, shape=(64, 64))
        volume_context = pad_or_crop_volume(context_slices, target_depth=7, shape=(256, 256))

        all_locals.append(volume_local)
        all_contexts.append(volume_context)
        all_labels.append(label)

    except Exception as e:
        print(f"Skipping row due to error: {e}")


Processing samples:  73%|███████▎  | 1058/1453 [00:13<00:05, 77.28it/s]

# 3. Radiomics preparation

In [ ]:
radiomics_columns = [
    "calcification", "internal_structure", "lobulation",
    "margin", "sphericity", "texture"
]

In [ ]:
radiomics_df = df[radiomics_columns].copy()

In [ ]:
radiomics_encoded = pd.get_dummies(radiomics_df, drop_first=False)

In [ ]:
print("Radiomics vector shape:", radiomics_encoded.shape)
radiomics_encoded.head()

In [ ]:
X_radiomics = radiomics_encoded.to_numpy(dtype=np.float32)

In [ ]:
X_radiomics.shape

# 4. Create Pytorch Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

In [ ]:
class TripleFusionDataset(Dataset):
    def __init__(self, all_locals, all_contexts, X_radiomics, labels_df):
        self.all_locals = all_locals
        self.all_contexts = all_contexts
        self.radiomics = X_radiomics
        self.labels = labels_df["label"].to_numpy()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {
            "volume_local": torch.tensor(self.all_locals[idx], dtype=torch.float32),
            "volume_context": torch.tensor(self.all_contexts[idx], dtype=torch.float32),
            "radiomics": torch.tensor(self.radiomics[idx], dtype=torch.float32),
            "label": torch.tensor(self.labels[idx], dtype=torch.long),
        }
        return sample


In [ ]:
dataset = TripleFusionDataset(
    all_locals=all_locals,
    all_contexts=all_contexts,
    X_radiomics=X_radiomics,
    labels_df=df  
)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# 5. Triple Fusion Architecture


| Branch           | Input Shape           | Type     |
|------------------|------------------------|----------|
| `volume_local`   | `[B, 1, 5, 64, 64]`     | 3D CNN   |
| `volume_context` | `[B, 1, 7, 256, 256]`   | 3D CNN   |
| `radiomics`      | `[B, 25]`               | MLP      |


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Local Branch

In [ ]:
class LocalBranch(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(2),  # (5,64,64) → (2,32,32)

            nn.Conv3d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d(2),

            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool3d((1, 1, 1))  # output: [B, 32, 1, 1, 1]
        )

    def forward(self, x):
        x = self.conv(x)
        return x.view(x.size(0), -1)  # flatten to [B, 32]

### Context Branch

In [ ]:
class ContextBranch(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(2),  # (7,256,256) → (3,128,128)

            nn.Conv3d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.AdaptiveAvgPool3d((1, 1, 1))  # output: [B, 32, 1, 1, 1]
        )

    def forward(self, x):
        x = self.conv(x)
        return x.view(x.size(0), -1)  # [B, 32]

### Radiomics Branch

In [ ]:
class RadiomicsBranch(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )

    def forward(self, x):
        return self.fc(x)

### Fusion Model

In [ ]:
class TripleFusionModel(nn.Module):
    def __init__(self, num_classes, radiomics_dim=25):
        super().__init__()
        self.local_branch = LocalBranch()
        self.context_branch = ContextBranch()
        self.radiomics_branch = RadiomicsBranch(radiomics_dim)

        self.classifier = nn.Sequential(
            nn.Linear(32 + 32 + 64, 64),  # fuse outputs
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, volume_local, volume_context, radiomics):
        local_feat = self.local_branch(volume_local)
        context_feat = self.context_branch(volume_context)
        radio_feat = self.radiomics_branch(radiomics)

        fused = torch.cat([local_feat, context_feat, radio_feat], dim=1)
        out = self.classifier(fused)
        return out

# 6. Training

In [ ]:
import torch.optim as optim

num_classes = len(df["label"].unique())
model = TripleFusionModel(num_classes=num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def compute_accuracy(logits, labels):
    preds = torch.argmax(logits, dim=1)
    return (preds == labels).float().mean().item()

In [ ]:
from tqdm import tqdm
import time

def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    total_acc = 0.0

    pbar = tqdm(dataloader, desc="Training", leave=False)

    for batch in pbar:
        start_time = time.time()

        vol_local = batch["volume_local"].to(device)
        vol_context = batch["volume_context"].to(device)
        radiomics = batch["radiomics"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(vol_local, vol_context, radiomics)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        acc = compute_accuracy(outputs, labels)
        batch_time = time.time() - start_time

        total_loss += loss.item()
        total_acc += acc

        pbar.set_postfix({
            "loss": f"{loss.item():.4f}",
            "acc": f"{acc:.4f}",
            "time": f"{batch_time:.2f}s"
        })

    return total_loss / len(dataloader), total_acc / len(dataloader)


In [ ]:
def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0

    pbar = tqdm(dataloader, desc="Validation", leave=False)

    with torch.no_grad():
        for batch in pbar:
            vol_local = batch["volume_local"].to(device)
            vol_context = batch["volume_context"].to(device)
            radiomics = batch["radiomics"].to(device)
            labels = batch["label"].to(device)

            outputs = model(vol_local, vol_context, radiomics)
            loss = criterion(outputs, labels)
            acc = compute_accuracy(outputs, labels)

            total_loss += loss.item()
            total_acc += acc

            pbar.set_postfix({
                "loss": f"{loss.item():.4f}",
                "acc": f"{acc:.4f}"
            })

    return total_loss / len(dataloader), total_acc / len(dataloader)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
from torch.utils.data import random_split, DataLoader

# Train-Validation Split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
val_loader = DataLoader(val_set, batch_size=8, shuffle=False)

In [ ]:
num_epochs = 40

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Accuracy: {train_acc:.4f}")
    print(f"  Val   Loss: {val_loss:.4f} | Accuracy: {val_acc:.4f}")

In [ ]:
num_epochs = 40

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Accuracy: {train_acc:.4f}")
    print(f"  Val   Loss: {val_loss:.4f} | Accuracy: {val_acc:.4f}")

# 7. Evaluation

NOTE: I forgot to do a test dataset split so I'm currently gonna get a random subset

In [ ]:
def predict(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            vol_local = batch["volume_local"].to(device)
            vol_context = batch["volume_context"].to(device)
            radiomics = batch["radiomics"].to(device)
            labels = batch["label"].to(device)

            outputs = model(vol_local, vol_context, radiomics)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_model(model, dataloader, device, class_names):
    y_pred, y_true = predict(model, dataloader, device)

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="weighted")

    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()


In [ ]:
test_set = torch.utils.data.Subset(dataset, range(int(0.85 * len(dataset)), len(dataset)))
test_loader = DataLoader(test_set, batch_size=8, shuffle=False)

In [ ]:
class_names = list(le.classes_)
evaluate_model(model, test_loader, device, class_names)

# 8. Explainability

### Grad-Cam

In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
target_layer = model.local_branch.conv[4]  # the second Conv3D layer

In [ ]:
cam = GradCAM(model=model.local_branch, target_layers=[target_layer])

batch = next(iter(test_loader))  # one batch
input_tensor = batch["volume_local"][0].unsqueeze(0).to(device)  # [1, 1, D, H, W]
target_category = batch["label"][0].item()

grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(target_category)])


In [ ]:
# Visualize the heatmap 

input_volume = batch["volume_local"][0].squeeze(0).cpu().numpy() 

def normalize(x):
    x = x - x.min()
    x = x / (x.max() + 1e-5)
    return x

volume = batch["volume_local"][0].squeeze(0).cpu().numpy()      
grayscale = grayscale_cam[0]                                    

# Take the middle slice
mid = volume.shape[0] // 2
slice_img = normalize(volume[mid])
cam_slice = normalize(grayscale[mid])

if cam_slice.shape != slice_img.shape:
    cam_slice = cv2.resize(cam_slice, slice_img.shape[::-1])

# Convert CAM to heatmap
cam_uint8 = np.ascontiguousarray((cam_slice * 255).astype(np.uint8))
heatmap = cv2.applyColorMap(cam_uint8, cv2.COLORMAP_JET)
heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB) / 255.0

# Prepare grayscale CT image
ct_rgb = np.stack([slice_img] * 3, axis=-1)

overlay = 0.5 * ct_rgb + 0.5 * heatmap

plt.figure(figsize=(6, 6))
plt.imshow(overlay)
plt.title("Grad-CAM Overlay (Middle Slice)")
plt.axis("off")
plt.show()

# 9. Save model

In [ ]:
torch.save(model.state_dict(), "triple_fusion_model.pkl")